In [1]:
import sys
sys.path.append('..')
from economic_brazil.visualizacoes_graficas.codigos_graficos import Graficos
from economic_brazil.coleta_dados.economic_data_brazil import data_economic
from economic_brazil.processando_dados.data_processing import criando_dummy_covid, criando_defasagens,criando_mes_ano_dia
from economic_brazil.processando_dados.estacionaridade import Estacionaridade
from economic_brazil.processando_dados.divisao_treino_teste import treino_test_dados,treino_teste_seies_temporal
from economic_brazil.treinamento.arima_treinamento import Arima



In [2]:
dados = data_economic()

In [32]:
valores_series = treino_teste_seies_temporal(dados,'selic',numero_divisoes=10,gap_series=20,max_train_size=None,test_size=10)

In [41]:
from sklearn.linear_model import LinearRegression
def treinar_regresao_linear(dados, series_divididas, target):
    y = dados[[target]]
    X = dados.drop(target, axis=1)
    lr_predictions, lr_scores = [], []
    lr = LinearRegression()
    for train_idx, test_idx in series_divididas[0:2]:
        X_train, y_train, = X.iloc[train_idx], y[target].iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y[target].iloc[test_idx]
        lr.fit(X=X_train, y=y_train)
        y_pred = lr.predict(X_test)

        preds = y_test.to_frame('actuals').assign(predicted=y_pred)
    return preds

In [43]:
teste_reger = treinar_regresao_linear(dados, valores_series, 'selic')

In [45]:

preds_by_day = teste_reger.groupby(level='Date')
scores = pd.concat([preds_by_day.apply(lambda x: spearmanr(x.predicted,
                                                                x.actuals)[0] * 100)
                            .to_frame('ic'),
                            preds_by_day.apply(lambda x: np.sqrt(mean_squared_error(y_pred=x.predicted,
                                                                                    y_true=x.actuals)))
                            .to_frame('rmse')], axis=1)

In [48]:
teste_reger

,actuals,predicted
Date,,
2016-11-01,13.90,6.776322
2016-12-01,13.65,5.807808
2017-01-01,13.17,7.104956
2017-02-01,12.82,7.162148
2017-03-01,12.15,7.112139
2017-04-01,11.59,5.859679
2017-05-01,11.15,7.593916
2017-06-01,10.15,5.918633
2017-07-01,10.01,5.796941


In [61]:
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import pandas as pd
import numpy as np

def treinar_regresao_linear(dados, series_divididas, target):
    y = dados[[target]]
    X = dados.drop(target, axis=1)
    lr_predictions, lr_scores = [], []
    lr = LinearRegression()
    for train_idx, test_idx in series_divididas:
        X_train, y_train, = X.iloc[train_idx], y[target].iloc[train_idx]
        X_test, y_test = X.iloc[test_idx], y[target].iloc[test_idx]
        lr.fit(X=X_train, y=y_train)
        y_pred = lr.predict(X_test)

        preds = y_test.to_frame('actuals').assign(predicted=y_pred)
        preds_by_day = preds.groupby(level='Date')
        scores = pd.concat([preds_by_day.apply(lambda x: spearmanr(x.predicted,
                                                                x.actuals)[0]*100)
                            .to_frame('ic'),
                            preds_by_day.apply(lambda x: np.sqrt(mean_squared_error(y_pred=x.predicted,
                                                                                    y_true=x.actuals)))
                            .to_frame('rmse')], axis=1)

        lr_scores.append(scores)
        lr_predictions.append(preds)

    lr_scores = pd.concat(lr_scores)
    lr_predictions = pd.concat(lr_predictions)
    return lr_scores, lr_predictions

In [65]:

# Realizar validação cruzada
y = dados[['selic']]
X = dados.drop('selic', axis=1)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

# Assuming X and y are your feature matrix and target variable
tscv = TimeSeriesSplit(n_splits=10)

# Instantiate your model (e.g., RandomForestRegressor)
model = RandomForestRegressor()

# Perform cross-validation
cv_scores = cross_val_score(model, X, y, cv=tscv, scoring='r2')

In [70]:
cv_scores

array([-4.43272786e-01,  3.41961011e-02, -1.54196258e+01, -5.86114289e+00,
       -8.30857530e-01, -1.51354879e+00,  4.44055752e-01, -4.72616059e+01,
       -9.45354228e-01, -4.21857785e+01])

In [62]:
lr_scores, lr_predictions = treinar_regresao_linear(dados, valores_series, 'selic')

In [63]:
lr_scores

,ic,rmse
Date,,
2016-01-01,NaN,4.864534
2016-02-01,NaN,5.574291
2016-03-01,NaN,7.420935
2016-04-01,NaN,7.235696
2016-05-01,NaN,5.445001
...,...,...
2023-12-01,NaN,5.478525
2024-01-01,NaN,5.744988
2024-02-01,NaN,6.529432


In [35]:
dados.iloc[valores_series[9][0]]

,selic,IPCA-EX2,IPCA-EX3,IPCA-MS,IPCA-MA,IPCA-EX0,IPCA-EX1,IPCA-DP,expectativas_inflacao,meta_inflacao,inflacao_efetiva,diferenca_meta_efetiva,ipca,pib,despesas_publica,capital_fixo,producao_industrial_manufatureira,mes,ano
Date,,,,,,,,,,,,,,,,,,,
2000-01-01,18.94,0.42,0.33,0.48,0.35,0.56,0.63,0.47,0.522857,6.00,5.97,0.03,0.62,4.4,3.6,-0.3,51.67275,1,2000
2000-02-01,18.87,0.17,0.21,0.44,0.35,0.45,0.32,0.32,0.531897,6.00,5.97,0.03,0.13,4.4,3.6,-0.3,51.67275,2,2000
2000-03-01,18.85,-0.11,-0.03,0.37,0.35,0.05,0.16,0.21,0.593452,6.00,5.97,0.03,0.22,4.4,3.6,-0.3,51.67275,3,2000
2000-04-01,18.62,0.16,0.22,0.45,0.35,0.87,0.62,0.60,0.587363,6.00,5.97,0.03,0.42,4.0,1.3,3.9,51.67275,4,2000
2000-05-01,18.51,0.12,0.16,0.35,0.35,0.20,0.19,0.18,0.562051,6.00,5.97,0.03,0.01,4.0,1.3,3.9,51.67275,5,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-01,3.76,0.59,0.55,0.58,0.55,0.45,0.54,0.50,0.325679,3.75,10.06,-6.31,0.53,12.4,8.1,26.7,90.79873,6,2021
2021-07-01,4.15,0.57,0.54,0.58,0.61,0.68,0.91,0.60,0.331931,3.75,10.06,-6.31,0.96,4.2,6.1,14.8,96.32546,7,2021
2021-08-01,5.01,0.67,0.62,0.68,0.70,0.64,0.53,0.70,0.336053,3.75,10.06,-6.31,0.87,4.2,6.1,14.8,95.55956,8,2021


In [ ]:
for i in range(len(valores_series)):
    train_idx, test_idx = valores_series[i]
    X_train, X_test = dados.iloc[train_idx], dados.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

In [6]:
dados_mes = criando_mes_ano_dia(dados,mes=True,dummy=True,coluns=['mes'])
dados_mes

,selic,IPCA-EX2,IPCA-EX3,IPCA-MS,IPCA-MA,IPCA-EX0,IPCA-EX1,IPCA-DP,expectativas_inflacao,meta_inflacao,...,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,mes_10,mes_11,mes_12
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,18.94,0.42,0.33,0.48,0.35,0.56,0.63,0.47,0.522857,6.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-01,18.87,0.17,0.21,0.44,0.35,0.45,0.32,0.32,0.531897,6.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-03-01,18.85,-0.11,-0.03,0.37,0.35,0.05,0.16,0.21,0.593452,6.00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-04-01,18.62,0.16,0.22,0.45,0.35,0.87,0.62,0.60,0.587363,6.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-05-01,18.51,0.12,0.16,0.35,0.35,0.20,0.19,0.18,0.562051,6.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,11.87,0.42,0.46,0.47,0.37,0.47,0.48,0.43,0.309085,3.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2024-01-01,11.65,0.59,0.61,0.51,0.37,0.16,0.22,0.45,0.308143,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-01,11.15,0.30,0.28,0.42,0.48,0.73,0.61,0.53,0.307386,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
import pandas as pd
def criando_mes_ano_dia(dados, mes=None, ano=None, dia=None, dummy=None, coluns=['year', 'month', 'day']):
    if mes:
        dados["mes"] = dados.index.month
    if ano:
        dados["ano"] = dados.index.year
    if dia:
        dados["dia"] = dados.index.day
    if dummy:
        dados_dia_mes_ano = pd.get_dummies(dados,
                                           columns=coluns,
                                           prefix=[i for i in coluns],
                                           prefix_sep='_',
                                           drop_first=True).astype(float)
        dados = dados_dia_mes_ano.copy()
    
    return dados


In [5]:
criando_mes_ano_dia(dados,mes=True,dummy=True,coluns=['mes'])

,selic,IPCA-EX2,IPCA-EX3,IPCA-MS,IPCA-MA,IPCA-EX0,IPCA-EX1,IPCA-DP,expectativas_inflacao,meta_inflacao,...,mes_3,mes_4,mes_5,mes_6,mes_7,mes_8,mes_9,mes_10,mes_11,mes_12
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,18.94,0.42,0.33,0.48,0.35,0.56,0.63,0.47,0.522857,6.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-02-01,18.87,0.17,0.21,0.44,0.35,0.45,0.32,0.32,0.531897,6.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-03-01,18.85,-0.11,-0.03,0.37,0.35,0.05,0.16,0.21,0.593452,6.00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-04-01,18.62,0.16,0.22,0.45,0.35,0.87,0.62,0.60,0.587363,6.00,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2000-05-01,18.51,0.12,0.16,0.35,0.35,0.20,0.19,0.18,0.562051,6.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,11.87,0.42,0.46,0.47,0.37,0.47,0.48,0.43,0.309085,3.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2024-01-01,11.65,0.59,0.61,0.51,0.37,0.16,0.22,0.45,0.308143,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-02-01,11.15,0.30,0.28,0.42,0.48,0.73,0.61,0.53,0.307386,3.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dados_dummy = criando_dummy_covid(dados, '2020-04-01', '2020-05-01')
dados_dummy

,selic,IPCA-EX2,IPCA-EX3,IPCA-MS,IPCA-MA,IPCA-EX0,IPCA-EX1,IPCA-DP,expectativas_inflacao,meta_inflacao,inflacao_efetiva,diferenca_meta_efetiva,ipca,pib,despesas_publica,capital_fixo,producao_industrial_manufatureira,dummy_covid
Date,,,,,,,,,,,,,,,,,,
2000-01-01,18.94,0.42,0.33,0.48,0.35,0.56,0.63,0.47,0.522857,6.00,5.97,0.03,0.62,4.4,3.6,-0.3,51.67275,0
2000-02-01,18.87,0.17,0.21,0.44,0.35,0.45,0.32,0.32,0.531897,6.00,5.97,0.03,0.13,4.4,3.6,-0.3,51.67275,0
2000-03-01,18.85,-0.11,-0.03,0.37,0.35,0.05,0.16,0.21,0.593452,6.00,5.97,0.03,0.22,4.4,3.6,-0.3,51.67275,0
2000-04-01,18.62,0.16,0.22,0.45,0.35,0.87,0.62,0.60,0.587363,6.00,5.97,0.03,0.42,4.0,1.3,3.9,51.67275,0
2000-05-01,18.51,0.12,0.16,0.35,0.35,0.20,0.19,0.18,0.562051,6.00,5.97,0.03,0.01,4.0,1.3,3.9,51.67275,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,11.87,0.42,0.46,0.47,0.37,0.47,0.48,0.43,0.309085,3.25,4.62,-1.37,0.56,2.1,3.0,-4.4,84.26269,0
2024-01-01,11.65,0.59,0.61,0.51,0.37,0.16,0.22,0.45,0.308143,3.00,4.62,-1.37,0.42,2.1,3.0,-4.4,84.26269,0
2024-02-01,11.15,0.30,0.28,0.42,0.48,0.73,0.61,0.53,0.307386,3.00,4.62,-1.37,0.83,2.1,3.0,-4.4,84.26269,0


In [ ]:
test_est = Estacionaridade()

In [ ]:
test_est.test_kpss_adf(dados=dados)

,Teste_kpss,Estacionaria_ou_nâo_kpss,Teste_adf,Estacionaria_ou_nâo_adf
selic,0.01,Nâo estacionaria,0.201426,Nâo estacionaria
IPCA-EX2,0.1,Estacionaria,0.005852,Estacionaria
IPCA-EX3,0.1,Estacionaria,0.007026,Estacionaria
IPCA-MS,0.1,Estacionaria,0.06079,Nâo estacionaria
IPCA-MA,0.1,Estacionaria,0.009707,Estacionaria
IPCA-EX0,0.1,Estacionaria,0.025688,Estacionaria
IPCA-EX1,0.1,Estacionaria,0.079144,Nâo estacionaria
IPCA-DP,0.1,Estacionaria,0.005241,Estacionaria
expectativas_inflacao,0.011041,Nâo estacionaria,0.234689,Nâo estacionaria
meta_inflacao,0.012034,Nâo estacionaria,0.061469,Nâo estacionaria


In [ ]:
esta_dif = test_est.report_ndiffs(dados=dados, test=["kpss", "adf", "pp"], alpha=0.05)
esta_dif

,kpss,adf,pp,Ndifis
selic,1,1,1,1
producao_industrial_manufatureira,1,1,1,1
diferenca_meta_efetiva,0,1,1,1
inflacao_efetiva,0,1,1,1
IPCA-EX2,0,0,0,0
capital_fixo,0,0,0,0
despesas_publica,1,0,0,0
pib,1,0,0,0
ipca,0,0,0,0
meta_inflacao,1,0,0,0


In [ ]:
test_est.corrigindo_nao_estacionaridade(dados,esta_dif,'selic')

,selic,IPCA-EX2,IPCA-EX3,IPCA-MS,IPCA-MA,IPCA-EX0,IPCA-EX1,IPCA-DP,expectativas_inflacao,meta_inflacao,inflacao_efetiva,diferenca_meta_efetiva,ipca,pib,despesas_publica,capital_fixo,producao_industrial_manufatureira,dummy_covid
Date,,,,,,,,,,,,,,,,,,
2000-02-01,18.87,0.17,0.21,0.44,0.35,0.45,0.32,0.32,0.531897,6.00,0.0,0.0,0.13,4.4,3.6,-0.3,0.0,0
2000-03-01,18.85,-0.11,-0.03,0.37,0.35,0.05,0.16,0.21,0.593452,6.00,0.0,0.0,0.22,4.4,3.6,-0.3,0.0,0
2000-04-01,18.62,0.16,0.22,0.45,0.35,0.87,0.62,0.60,0.587363,6.00,0.0,0.0,0.42,4.0,1.3,3.9,0.0,0
2000-05-01,18.51,0.12,0.16,0.35,0.35,0.20,0.19,0.18,0.562051,6.00,0.0,0.0,0.01,4.0,1.3,3.9,0.0,0
2000-06-01,18.04,0.34,0.36,0.42,0.35,0.30,0.31,0.28,0.579477,6.00,0.0,0.0,0.23,4.0,1.3,3.9,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,11.87,0.42,0.46,0.47,0.37,0.47,0.48,0.43,0.309085,3.25,0.0,0.0,0.56,2.1,3.0,-4.4,0.0,0
2024-01-01,11.65,0.59,0.61,0.51,0.37,0.16,0.22,0.45,0.308143,3.00,0.0,0.0,0.42,2.1,3.0,-4.4,0.0,0
2024-02-01,11.15,0.30,0.28,0.42,0.48,0.73,0.61,0.53,0.307386,3.00,0.0,0.0,0.83,2.1,3.0,-4.4,0.0,0


In [ ]:
criando_defasagens(dados)

,selic,IPCA-EX2,IPCA-EX3,IPCA-MS,IPCA-MA,IPCA-EX0,IPCA-EX1,IPCA-DP,expectativas_inflacao,meta_inflacao,...,expectativas_inflacao_lags_4,meta_inflacao_lags_4,inflacao_efetiva_lags_4,diferenca_meta_efetiva_lags_4,ipca_lags_4,pib_lags_4,despesas_publica_lags_4,capital_fixo_lags_4,producao_industrial_manufatureira_lags_4,dummy_covid_lags_4
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-01,18.94,0.42,0.33,0.48,0.35,0.56,0.63,0.47,0.522857,6.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-02-01,18.87,0.17,0.21,0.44,0.35,0.45,0.32,0.32,0.531897,6.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-03-01,18.85,-0.11,-0.03,0.37,0.35,0.05,0.16,0.21,0.593452,6.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-04-01,18.62,0.16,0.22,0.45,0.35,0.87,0.62,0.60,0.587363,6.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-05-01,18.51,0.12,0.16,0.35,0.35,0.20,0.19,0.18,0.562051,6.00,...,0.522857,6.00,5.97,0.03,0.62,4.4,3.6,-0.3,51.67275,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-01,11.87,0.42,0.46,0.47,0.37,0.47,0.48,0.43,0.309085,3.25,...,0.315296,3.25,4.62,-1.37,0.23,2.0,0.8,-6.8,84.26269,0.0
2024-01-01,11.65,0.59,0.61,0.51,0.37,0.16,0.22,0.45,0.308143,3.00,...,0.317083,3.25,4.62,-1.37,0.26,2.0,0.8,-6.8,84.26269,0.0
2024-02-01,11.15,0.30,0.28,0.42,0.48,0.73,0.61,0.53,0.307386,3.00,...,0.312520,3.25,4.62,-1.37,0.24,2.1,3.0,-4.4,84.26269,0.0


In [ ]:
arima = Arima()

In [ ]:
model = arima.treinar_arima(dados=dados.selic)

                               SARIMAX Results                                
Dep. Variable:                  selic   No. Observations:                  292
Model:                 ARIMA(1, 0, 1)   Log Likelihood                -108.899
Date:                Sun, 28 Apr 2024   AIC                            225.798
Time:                        20:10:17   BIC                            240.505
Sample:                    01-01-2000   HQIC                           231.689
                         - 04-01-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.4247      3.416      3.929      0.000       6.729      20.121
ar.L1          0.9928      0.006    179.884      0.000       0.982       1.004
ma.L1          0.7098      0.027     26.285      0.0

In [ ]:
best_param,p,q = arima.encontrar_parametros_arima(dados=dados.selic,divisao=240)

In [ ]:
modelo_arima = arima.treinar_arima(dados=dados.selic,p=p,q=q)
predicao = arima.prever_arima(modelo=modelo_arima,steps=5)
predicao

                               SARIMAX Results                                
Dep. Variable:                  selic   No. Observations:                  292
Model:                 ARIMA(2, 0, 4)   Log Likelihood                 -35.269
Date:                Sun, 28 Apr 2024   AIC                             86.538
Time:                        20:15:05   BIC                            115.952
Sample:                    01-01-2000   HQIC                            98.320
                         - 04-01-2024                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.5520      2.129      5.897      0.000       8.380      16.724
ar.L1          1.7658      0.072     24.633      0.000       1.625       1.906
ar.L2         -0.7749      0.070    -11.032      0.0

2024-05-01    10.329823
2024-06-01    10.168589
2024-07-01     9.985643
2024-08-01     9.894329
2024-09-01     9.847948
Freq: MS, Name: predicted_mean, dtype: float64

In [ ]:
graficos = Graficos()
graficos.plot_correlogram_colunas(dados=dados, lags_correlogram=40,  save_correlogram=True, diretorio_correlogram="/workspaces/Predicoes_macroeconomicas/imagens")

<Figure size 640x480 with 0 Axes>

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_openml

bike_sharing = fetch_openml(
    "Bike_Sharing_Demand", version=2, as_frame=True, parser="pandas"
)
df = bike_sharing.frame

In [ ]:
summary = pd.DataFrame(df.describe())
summary = (
    summary.style.background_gradient()
    .set_table_attributes("style = 'display: inline'")
    .set_caption("Statistics of the Dataset")
    .set_table_styles([{"selector": "caption", "props": [("font-size", "16px")]}])
)
summary

,year,month,hour,weekday,temp,feel_temp,humidity,windspeed,count
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000
mean,0.502561,6.537775,11.546752,3.003683,20.376474,23.788755,0.627229,12.736540,189.463088
std,0.500008,3.438776,6.914405,2.005771,7.894801,8.592511,0.192930,8.196795,181.387599
min,0.000000,1.000000,0.000000,0.000000,0.820000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,4.000000,6.000000,1.000000,13.940000,16.665000,0.480000,7.001500,40.000000
50%,1.000000,7.000000,12.000000,3.000000,20.500000,24.240000,0.630000,12.998000,142.000000
75%,1.000000,10.000000,18.000000,5.000000,27.060000,31.060000,0.780000,16.997900,281.000000
max,1.000000,12.000000,23.000000,6.000000,41.000000,50.000000,1.000000,56.996900,977.000000


In [ ]:
count = df["count"]
lagged_df = pd.concat(
    [
        count,
        count.shift(1).rename("lagged_count_1h"),
        count.shift(2).rename("lagged_count_2h"),
        count.shift(3).rename("lagged_count_3h"),
        count.shift(24).rename("lagged_count_1d"),
        count.shift(24 + 1).rename("lagged_count_1d_1h"),
        count.shift(7 * 24).rename("lagged_count_7d"),
        count.shift(7 * 24 + 1).rename("lagged_count_7d_1h"),
        count.shift(1).rolling(24).mean().rename("lagged_mean_24h"),
        count.shift(1).rolling(24).max().rename("lagged_max_24h"),
        count.shift(1).rolling(24).min().rename("lagged_min_24h"),
        count.shift(1).rolling(7 * 24).mean().rename("lagged_mean_7d"),
        count.shift(1).rolling(7 * 24).max().rename("lagged_max_7d"),
        count.shift(1).rolling(7 * 24).min().rename("lagged_min_7d"),
    ],
    axis="columns",
)
lagged_df.tail(10)

,count,lagged_count_1h,lagged_count_2h,lagged_count_3h,lagged_count_1d,lagged_count_1d_1h,lagged_count_7d,lagged_count_7d_1h,lagged_mean_24h,lagged_max_24h,lagged_min_24h,lagged_mean_7d,lagged_max_7d,lagged_min_7d
17369,247,203.0,224.0,157.0,160.0,169.0,70.0,135.0,93.500000,224.0,1.0,67.732143,271.0,1.0
17370,315,247.0,203.0,224.0,138.0,160.0,46.0,70.0,97.125000,247.0,1.0,68.785714,271.0,1.0
17371,214,315.0,247.0,203.0,133.0,138.0,33.0,46.0,104.500000,315.0,1.0,70.386905,315.0,1.0
17372,164,214.0,315.0,247.0,123.0,133.0,33.0,33.0,107.875000,315.0,1.0,71.464286,315.0,1.0
17373,122,164.0,214.0,315.0,125.0,123.0,26.0,33.0,109.583333,315.0,1.0,72.244048,315.0,1.0
17374,119,122.0,164.0,214.0,102.0,125.0,26.0,26.0,109.458333,315.0,1.0,72.815476,315.0,1.0
17375,89,119.0,122.0,164.0,72.0,102.0,18.0,26.0,110.166667,315.0,1.0,73.369048,315.0,1.0
17376,90,89.0,119.0,122.0,47.0,72.0,23.0,18.0,110.875000,315.0,1.0,73.791667,315.0,1.0
17377,61,90.0,89.0,119.0,36.0,47.0,22.0,23.0,112.666667,315.0,1.0,74.190476,315.0,1.0
17378,49,61.0,90.0,89.0,49.0,36.0,12.0,22.0,113.708333,315.0,1.0,74.422619,315.0,1.0


In [ ]:
lagged_df = lagged_df.dropna()
X = lagged_df.drop("count", axis="columns")
y = lagged_df["count"]
print("X shape: {}\ny shape: {}".format(X.shape, y.shape))

X shape: (17210, 13)
y shape: (17210,)


In [ ]:
from sklearn.model_selection import TimeSeriesSplit

ts_cv = TimeSeriesSplit(
    n_splits=3,  # to keep the notebook fast enough on common laptops
    gap=48,  # 2 days data gap between train and test
    max_train_size=10000,  # keep train sets of comparable sizes
    test_size=3000,  # for 2 or 3 digits of precision in scores
)
all_splits = list(ts_cv.split(X, y))

In [ ]:
train_idx, test_idx = all_splits[0]
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]


In [ ]:
X_train

,lagged_count_1h,lagged_count_2h,lagged_count_3h,lagged_count_1d,lagged_count_1d_1h,lagged_count_7d,lagged_count_7d_1h,lagged_mean_24h,lagged_max_24h,lagged_min_24h,lagged_mean_7d,lagged_max_7d,lagged_min_7d
169,9.0,2.0,5.0,210.0,84.0,40.0,16.0,60.083333,210.0,1.0,56.416667,219.0,1.0
170,15.0,9.0,2.0,134.0,210.0,32.0,40.0,51.958333,187.0,1.0,56.267857,219.0,1.0
171,20.0,15.0,9.0,63.0,134.0,13.0,32.0,47.208333,187.0,1.0,56.196429,219.0,1.0
172,61.0,20.0,15.0,67.0,63.0,1.0,13.0,47.125000,187.0,1.0,56.482143,219.0,1.0
173,62.0,61.0,20.0,59.0,67.0,1.0,1.0,46.916667,187.0,1.0,56.845238,219.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8326,228.0,228.0,188.0,237.0,236.0,242.0,268.0,106.208333,237.0,1.0,137.136905,467.0,1.0
8327,219.0,228.0,228.0,228.0,237.0,266.0,242.0,105.458333,228.0,1.0,137.000000,467.0,1.0
8328,207.0,219.0,228.0,222.0,228.0,206.0,266.0,104.583333,228.0,1.0,136.648810,467.0,1.0
8329,234.0,207.0,219.0,165.0,222.0,163.0,206.0,105.083333,234.0,1.0,136.815476,467.0,1.0
